In [218]:
import pandas as pd
import panel as pn # Panel is an open-source Python library that lets you create custom interactive web apps and dashboards by connecting user-defined widgets to plots, images, tables, or text.
pn.extension('tabulator')

import hvplot.pandas # By replacing .plot() with .hvplot() you get an interactive figure.

In [219]:
attrition = pd.read_csv('Attrition Data.csv')
attrition['id'] = attrition.index

In [220]:
attrition.head()

,Age,Attrition,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,WorkLifeBalance,YearsAtCompany,id
0,41,Yes,Sales,1,2,Life Sciences,2,4,Single,5993,8,1,6,0
1,49,No,Research & Development,8,1,Life Sciences,3,2,Married,5130,1,3,10,1
2,37,Yes,Research & Development,2,2,Other,4,3,Single,2090,6,3,0,2
3,33,No,Research & Development,3,4,Life Sciences,4,3,Married,2909,1,3,8,3
4,27,No,Research & Development,2,1,Medical,1,2,Married,3468,9,3,2,4


## data wrangling

In [221]:
education_map = {1: 'Below College', 2: 'College', 3: 'Bachelor', 4: 'Master', 5: 'Doctor'}
environment_map = {1: 'Low', 2: 'Medium', 3: 'High', 4: 'Very High'}
job_satisfaction_map = {1: 'Low', 2: 'Medium', 3: 'High', 4: 'Very High'}
work_life_balance_map = {1: 'Bad', 2: 'Good', 3: 'Better', 4: 'Best'}

In [222]:
attrition = attrition.replace({"Education": education_map})
attrition = attrition.replace({"EnvironmentSatisfaction": environment_map})
attrition = attrition.replace({"JobSatisfaction": job_satisfaction_map})
attrition = attrition.replace({"WorkLifeBalance": work_life_balance_map})

In [223]:
i_attrition = attrition.interactive()

## age slider

In [224]:
age_slider = pn.widgets.IntSlider(name='Age', start=20, end=60, step=1, value=40)
age_slider

IntSlider(end=60, name='Age', start=20, value=40)

In [225]:
range_slider = pn.widgets.RangeSlider(name='Range Slider', start=20, end=80, value=(25, 40), step=1)
range_slider

RangeSlider(end=80, name='Range Slider', start=20, step=1, value=(25, 40), value_end=40, value_start=25)

## department check boxes

In [226]:
department_check_boxes = pn.widgets.CheckBoxGroup(
    name = 'Department',
    value = ['Sales'],
    options = [opt for opt in attrition['Department'].unique()]
)
department_check_boxes

CheckBoxGroup(name='Department', options=['Sales', 'Research & Deve...], value=['Sales'])

## environment satisfaction check boxes

In [227]:
environment_satisfaction_checkbox = pn.widgets.CheckBoxGroup(
    name='Environment Satisfaction',
    value=['Medium', 'High'],
    options=[opt for opt in attrition['EnvironmentSatisfaction'].unique()],
    inline=False)
environment_satisfaction_checkbox

CheckBoxGroup(name='Environment S..., options=['Medium', 'High', ...], value=['Medium', 'High'])

## job satisfaction check boxes

In [228]:
job_satisfaction_checkbox = pn.widgets.CheckBoxGroup(
    name='Job Satisfaction',
    value=['High','Low'],
    options=[opt for opt in attrition['JobSatisfaction'].unique()],
    inline=False)
job_satisfaction_checkbox

CheckBoxGroup(name='Job Satisfaction', options=['Very High', ...], value=['High', 'Low'])

## marital status check boxes

In [229]:
marital_status_checkbox = pn.widgets.CheckBoxGroup(
    name='Marital Status',
    value=['Single'],
    options=[opt for opt in attrition['MaritalStatus'].unique()],
    inline=False)
marital_status_checkbox

CheckBoxGroup(name='Marital Status', options=['Single', 'Married', ...], value=['Single'])

## education drop down

In [230]:
education_drop_down = pn.widgets.CheckBoxGroup(
    name = 'Education',
    value = ['Bachelor', 'College'],
    options = [opt for opt in attrition.sort_values(by='Education')['Education'].unique()]
)
education_drop_down

CheckBoxGroup(name='Education', options=['Bachelor', 'Below Colleg...], value=['Bachelor', 'College'])

# visualizations

In [231]:
i_attrition = i_attrition[
    (i_attrition['Age']<=age_slider) &
    (i_attrition['Education'].isin(education_drop_down)) &
    (i_attrition['MaritalStatus'].isin(marital_status_checkbox)) &
    (i_attrition['EnvironmentSatisfaction'].isin(environment_satisfaction_checkbox)) &
    (i_attrition['JobSatisfaction'].isin(job_satisfaction_checkbox)) &
    (i_attrition['Department'].isin(department_check_boxes))
    ]
height = 400
width = 750

## viz 1: years at the company bar chart

In [232]:
years_at_company = (
    i_attrition
    .groupby(['YearsAtCompany', 'Attrition'])['id']
    .count()
    .reset_index()
    .pivot_table(index='YearsAtCompany', columns='Attrition', values='id')
    .reset_index()
)
i_years_at_company = years_at_company.hvplot(x='YearsAtCompany', y=['No', 'Yes'], height=height, width=width, legend='top_right', title='Count by years working at the company')

## viz 2: work life balance bar chart

In [233]:
work_life_pipeline = (
    i_attrition
    .groupby(['WorkLifeBalance', 'Attrition'])['id']
    .count()
)
i_work_life_pipeline = work_life_pipeline.hvplot.bar(stacked=True, height=height, legend='top_right', width=width, title='Count by work life balance')

## viz 3: Number of companies worked bar

In [234]:
companies_pipeline = (
    i_attrition
    .groupby(['NumCompaniesWorked', 'Attrition'])['id']
    .count()
)
i_companies_pipeline = companies_pipeline.hvplot.bar(stacked=True, height=height, legend='top_right', width=width, title='Count by number of companies worked for')

## viz 4: Distance from home bar

In [235]:
home_distance_pipeline = (
    i_attrition
    .groupby(['DistanceFromHome', 'Attrition'])['id']
    .count()
)
i_home_distance_pipeline = home_distance_pipeline.hvplot.bar(stacked=True, height=height, legend='top_right', width=width, title = 'Count by distance from home')

# putting it all together

In [236]:
text = """
Our client is ABC a leading firm and is doing well in the sector.
It is recently facing a steep increase in its employee attrition.
Employee attrition has gone up from 14% to 25% in the last 1 year.
We are asked to prepare a strategy to immediately tackle this issue
such that the firm’s business is not hampered and also to propose an
efficient employee satisfaction program for long run. Currently, no
such program is in place . Further salary hikes are not an option.
"""

template = pn.template.FastListTemplate(
    site="DaliCodes", title="Employee Attrition Dashboard",
    sidebar=[
        pn.pane.Markdown("## Settings"),
        age_slider,
        pn.pane.Markdown("Department"),
        department_check_boxes,
        pn.pane.Markdown("Education"),
        education_drop_down,
        pn.pane.Markdown("Marital Status"),
        marital_status_checkbox,
        pn.pane.Markdown("Environment Satisfaction"),
        environment_satisfaction_checkbox,
        pn.pane.Markdown("Job Satisfaction"),
        job_satisfaction_checkbox,
        pn.pane.Markdown("## Project Description"),
        pn.pane.PNG('attrition.png', sizing_mode='scale_both'),
        pn.pane.Markdown(text),
            ],
    main=[
        pn.Row(
            pn.Column(i_years_at_company.panel()),
            pn.Column(i_work_life_pipeline.panel())
        ),
        pn.Row(
            pn.Column(i_companies_pipeline.panel()),
            pn.Column(i_home_distance_pipeline.panel())
        )]
).servable();